<a href="https://colab.research.google.com/github/sarthak-somani/SOC-2025-Morphix/blob/main/Sarthak_Somani_SOC_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 38.59 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
!cd stylegan2-ada-pytorch

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.6.0
    Uninstalling imageio-ffmpeg-0.6.0:
      Successfully uninstalled imageio-ffmpeg-0.6.0


In [ ]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

--2025-06-25 17:15:53--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 13.35.37.10, 13.35.37.106, 13.35.37.115, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|13.35.37.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381624121 (364M) [binary/octet-stream]
Saving to: ‘ffhq.pkl’

ffhq.pkl            100%[===================>] 363.94M   195MB/s    in 1.9s    

2025-06-25 17:15:55 (195 MB/s) - ‘ffhq.pkl’ saved [381624121/381624121]



In [ ]:
!python /content/stylegan2-ada-pytorch/generate.py --outdir=out --trunc=1 --seeds=2,12,42 --network=ffhq.pkl

Loading networks from "ffhq.pkl"...
Generating image for seed 2 (0/3) ...
Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Generating image for seed 12 (1/3) ...
Generating image for seed 42 (2/3) ...


In [ ]:
import numpy as np
import torch
import pickle
import os

# --- Step 1: Set up the environment and device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# --- Step 2: Load the Pre-trained StyleGAN2-ADA Model ---
model_path = '../ffhq.pkl'
print(f'Loading networks from "{model_path}"...')

# Check if the model file exists before attempting to load it.
if not os.path.exists(model_path):
    print(f"Error: Model file not found at '{model_path}'.")
    print("Please ensure the path is correct and you have downloaded the model.")
else:
    # Load the generator network from the pickle file.
    # We load it into memory and then move the model to the selected device (GPU/CPU).
    # The 'G_ema' key holds the Exponential Moving Average of the generator's weights
    with open(model_path, 'rb') as f:
        G = pickle.load(f)['G_ema'].to(device)
    print('Model loaded successfully.')

    # Set the model to evaluation mode. This disables layers like dropout.
    G.eval()

    # --- Step 3: Generate a random Z vector ---

    # G.z_dim is the dimension of the Z space for the loaded model (typically 512).
    # The shape will be [1, 512], where 1 is the batch size.
    z_latent = np.random.randn(1, G.z_dim)

    # Convert the NumPy array to a PyTorch tensor and move it to the configured device.
    z_tensor = torch.from_numpy(z_latent).float().to(device)

    # --- Step 4: Map Z to W+ Space ---
    # We pass the Z tensor through the generator's mapping network.
    # This model's mapping network directly produces the W+ vector, which contains
    # a separate W vector for each layer of the synthesis network.
    print('Mapping Z vector directly to W+ space...')
    with torch.no_grad():
        # The resulting shape should be [1, num_layers, 512], e.g., [1, 18, 512].
        w_plus_latent = G.mapping(z_tensor, None)

    # --- Step 5: Derive W from W+ for Saving ---
    # The "conceptual" W space vector ([1, 512]) can be derived from W+.
    # For simplicity, we'll take the style vector for the first layer.
    # The previous logic that repeated the vector was incorrect for this model and has been removed.
    w_latent = w_plus_latent[:, 0, :]

    # --- Verification and Summary ---
    # Check the shapes to confirm they match the desired dimensions.
    print("\n--- Corrected Latent Vector Shapes ---")
    print(f"Z  (initial latent):  {z_tensor.shape}")
    print(f"W  (derived latent):   {w_latent.shape}")
    print(f"W+ (direct from map): {w_plus_latent.shape}")
    print("--------------------------------------")

    # --- Step 6: Save the W and W+ vectors ---
    # These .npy files can be used by other scripts for image generation or style mixing.
    w_numpy = w_latent.cpu().numpy()
    np.save('latent_w_01.npy', w_numpy)

    w_plus_numpy = w_plus_latent.cpu().numpy()
    np.save('latent_w_plus_01.npy', w_plus_numpy)

    print("\nSuccessfully saved 'latent_w_01.npy' and 'latent_w_plus_01.npy'.")
    print("These files are now ready for use in a generation script.")


Using device: cuda
Loading networks from "../ffhq.pkl"...
Model loaded successfully.
Mapping Z vector directly to W+ space...

--- Corrected Latent Vector Shapes ---
Z  (initial latent):  torch.Size([1, 512])
W  (derived latent):   torch.Size([1, 512])
W+ (direct from map): torch.Size([1, 18, 512])
--------------------------------------

Successfully saved 'latent_w_01.npy' and 'latent_w_plus_01.npy'.
These files are now ready for use in a generation script.


In [ ]:
import numpy as np
import torch
import pickle
import os
from PIL import Image

# --- Step 1: Set up the environment and device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# --- Step 2: Load the Pre-trained StyleGAN2-ADA Model ---
model_path = '../ffhq.pkl'
print(f'Loading networks from "{model_path}"...')

# Check if the model file exists before proceeding
if not os.path.exists(model_path):
    print("Error: Model file not found. Make sure 'ffhq.pkl' is in the parent directory.")
else:
    with open(model_path, 'rb') as f:
        # Load the generator network and move it to the correct device
        G = pickle.load(f)['G_ema'].to(device)
    print('Model loaded successfully.')
    G.eval()

    # --- Step 3: Define a helper function to convert tensor to image ---
    def tensor_to_pil(tensor):
        """
        Converts a PyTorch tensor (in NCHW format) to a PIL Image.
        The tensor is expected to be in the range [-1, 1].
        """
        # Denormalize from [-1, 1] to [0, 255]
        tensor = (tensor.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        # Convert to a PIL image from the first item in the batch
        return Image.fromarray(tensor[0].cpu().numpy(), 'RGB')

    # --- File paths for your saved latent vectors ---
    w_vector_path = 'latent_w_01.npy'
    w_plus_vector_path = 'latent_w_plus_01.npy'

    # --- Check if the latent vector files exist before trying to load them ---
    if not os.path.exists(w_vector_path) or not os.path.exists(w_plus_vector_path):
         print(f"Error: Latent vector files not found. Please create them first.")
    else:
        # --- Method A: Generate Image from the saved W vector ---
        print("\nGenerating from the standard W vector...")
        w_numpy = np.load(w_vector_path)
        w_tensor = torch.from_numpy(w_numpy).to(device)

        # The synthesis network ALWAYS expects a 3D W+ tensor of shape [batch, num_ws, w_dim].
        # We must prepare the tensor accordingly.
        # This is a correctly-shaped W vector, e.g., [1, 512], so we expand it.
        print(f"  Loaded W vector has shape: {w_tensor.shape}")
        w_for_synthesis = w_tensor.unsqueeze(1).repeat([1, G.num_ws, 1])
        print(f"  Expanded to W+ shape for synthesis: {w_for_synthesis.shape}")

        with torch.no_grad():
            img_from_w = G.synthesis(w_for_synthesis, noise_mode='const')

        pil_img_from_w = tensor_to_pil(img_from_w)
        pil_img_from_w.save('generated_from_W.png')
        print(f"-> Successfully saved 'generated_from_W.png'.")

        # --- Method B: Generate Image from the saved W+ vector ---
        print("\nGenerating from the W+ vector...")
        w_plus_numpy = np.load(w_plus_vector_path)
        w_plus_tensor = torch.from_numpy(w_plus_numpy).to(device)
        print(f"  Loaded W+ vector has correct shape: {w_plus_tensor.shape}")

        with torch.no_grad():
            img_from_w_plus = G.synthesis(w_plus_tensor, noise_mode='const')

        pil_img_from_w_plus = tensor_to_pil(img_from_w_plus)
        pil_img_from_w_plus.save('generated_from_W_plus.png')
        print(f"-> Successfully saved 'generated_from_W_plus.png'.")

        # The two generated images should now be identical.
        print("\nGeneration complete. The two images, 'generated_from_W.png' and 'generated_from_W_plus.png', should look the same.")


Using device: cuda
Loading networks from "../ffhq.pkl"...
Model loaded successfully.

Generating from the standard W vector...
  Loaded W vector has shape: torch.Size([1, 512])
  Expanded to W+ shape for synthesis: torch.Size([1, 18, 512])
-> Successfully saved 'generated_from_W.png'.

Generating from the W+ vector...
  Loaded W+ vector has correct shape: torch.Size([1, 18, 512])
-> Successfully saved 'generated_from_W_plus.png'.

Generation complete. The two images, 'generated_from_W.png' and 'generated_from_W_plus.png', should look the same.
